In [2]:
!pip install gradio

In [3]:
pip install openai==0.28.0

In [2]:
pip install transformers sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 26.7 MB/s eta 0:00:00


In [4]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Initialize the Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example medical records (in a real application, this would be a database or a collection of documents)
medical_records = [
    "Patient has a history of chest pain and shortness of breath.",
    "Recent ECG shows ST elevation indicating potential myocardial infarction.",
    "Prescribed medications include beta-blockers and aspirin.",
    "Patient history shows diabetes and hypertension.",
    "Patient presents with rashes on arms and legs, suspected of eczema."
]

# Convert the medical records into embeddings
embeddings = model.encode(medical_records)

# Build a FAISS index for efficient search
index = faiss.IndexFlatL2(embeddings.shape[1])  # Using L2 distance (Euclidean distance)
index.add(np.array(embeddings))
print(index)

# Function to retrieve the top-k most relevant medical records based on the query
def retrieve_relevant_documents(query, k=3):
    query_embedding = model.encode([query])  # Encode the query into the same embedding space
    D, I = index.search(np.array(query_embedding), k)  # Perform the search (D = distances, I = indices)

    relevant_docs = [medical_records[i] for i in I[0]]
    print(relevant_docs )
    return relevant_docs


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7ed221b223a0> >


In [14]:
import openai
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

def generate_summary(retrieved_docs, specialty):
    # Join the relevant documents retrieved by the retriever
    context = "\n".join(retrieved_docs)

    # Define the prompt to summarize the context
    prompt = f"Summarize the following medical record for a busy {specialty} physician. Focus on the important diagnostic findings and treatment plan.\n\n{context}"

    # Request GPT-3.5 (or GPT-4) to generate the summary
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use gpt-3.5-turbo or gpt-4
        messages=[
            {"role": "system", "content": "You are a medical assistant summarizing patient records for a physician."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.7
    )

    # Extract and return the summary
    return response['choices'][0]['message']['content'].strip()


Note: using Google CoLab


In [15]:
def generate_medical_summary(query, specialty="cardiology"):
    # Step 1: Retrieve relevant documents based on the query
    retrieved_docs = retrieve_relevant_documents(query, k=3)

    # Step 2: Generate the summary using GPT-3
    summary = generate_summary(retrieved_docs, specialty)

    return summary

# Example usage
query = "Patient complains of chest pain and shortness of breath."
specialty = "cardiology"
summary = generate_medical_summary(query, specialty)

print("Generated Summary:")
print(summary)


['Patient has a history of chest pain and shortness of breath.', 'Recent ECG shows ST elevation indicating potential myocardial infarction.', 'Patient history shows diabetes and hypertension.']
Generated Summary:
Patient with a history of chest pain and shortness of breath presented with ST elevation on recent ECG indicating potential myocardial infarction. Patient also has a history of diabetes and hypertension. Treatment plan should include immediate evaluation for possible myocardial infarction and management of diabetes and hypertension to reduce cardiovascular risk factors.


In [16]:
# Function to handle chatbot interaction
def chatbot(query, specialty="cardiology", chat_history=[]):
    # Retrieve relevant documents
    retrieved_docs = retrieve_relevant_documents(query)

    # Generate summary from the retrieved documents
    summary = generate_summary(retrieved_docs, specialty)

    # Update chat history
    chat_history.append(("Physician", query))
    chat_history.append(("Bot", summary))
    return chat_history, chat_history

# Gradio chatbot interface
with gr.Blocks() as demo:
    gr.Markdown("# Medical Records Summarization Chatbot")

    with gr.Row():
        chatbot_box = gr.Chatbot(label="Chat with Medical Summarizer")

    with gr.Row():
        query = gr.Textbox(
            label="Your Query",
            placeholder="Enter query, e.g., 'Patient complains of chest pain and shortness of breath'",
        )
        specialty = gr.Textbox(label="Specialty", placeholder="E.g., cardiology", value="cardiology")
        send_button = gr.Button("Send")

    # Interaction
    chat_history = gr.State([])
    send_button.click(
        chatbot,
        inputs=[query, specialty, chat_history],
        outputs=[chatbot_box, chat_history],
    )

# Launch Gradio app with a public link
demo.launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://65360ad6180923e77b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
